<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_9_mixed_pubmed_cadec_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,3,cracking sensations in my joints
1,5,fast and slow heartbeat
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-9.test.csv",header=None)
test.head()

,0,1
0,2,painful heels
1,4,Discoloration of skin [blotchy] on the neck
2,5,my heart rate jumped to 106
3,7,goiter
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_cadec_askpatient_mixed.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'muscle',
 'a',
 '.',
 'was',
 'with',
 'i',
 'diclofenac',
 ',',
 'my',
 'for',
 'were',
 'severe',
 'on',
 'lower',
 'myalgia',
 'fatigue',
 'loss',
 'is',
 'weakness',
 '-',
 'that',
 'by',
 'sodium',
 'cramp',
 'limb',
 'back',
 'stomach',
 'at',
 'as',
 'this',
 'drug',
 'depression',
 'not',
 'it',
 'cramps',
 'mg',
 'or',
 'patients',
 'joint',
 'leg',
 'gas',
 'upper',
 'memory',
 'from',
 'legs',
 'after',
 'pains',
 'all',
 'knee',
 'have',
 'headache',
 'shoulder',
 'unable',
 'group',
 '/',
 'neck',
 'arthralgia',
 'gastrointestinal',
 'be',
 'treatment',
 'study',
 'foot',
 'aches',
 'excessive',
 'arthritis',
 'like',
 'cholesterol',
 'no',
 'hip',
 'an',
 'had',
 'effects',
 'voltaren',
 'insomnia',
 'arthrotec',
 'numbness',
 'but',
 'time',
 'disease',
 'p',
 'walk',
 'are',
 'increased',
 'heart',
 'problems',
 'release',

In [22]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [23]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.816972,3.461866,0.436776,35:58


In [25]:
# learn.fit_one_cycle(1, 1e-2)

In [26]:
# learn.fit_one_cycle(1, 1e-3)

In [27]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.485849,3.406280,0.441987,36:09


In [28]:
learn.save_encoder('pubmed_cadec_askpatient_mixed_fold9_first')

In [29]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_askpatient_mixed_fold9_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.958101,3.319452,0.423181,01:11


In [30]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.875328,2.739612,0.517520,01:28


In [31]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.339214,2.345653,0.597035,03:22


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.992968,2.085386,0.640162,03:19
1,1.702852,1.956919,0.642857,03:19


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.597936,1.768976,0.684636,03:42
1,1.320490,1.679849,0.703504,03:17


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.177221,1.555274,0.725067,03:32
1,1.027568,1.490998,0.742588,03:16


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.970472,1.415296,0.746631,03:40
1,0.844565,1.373346,0.753369,03:27


In [36]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.808211,1.331710,0.760108,03:34
1,0.699164,1.278810,0.787062,03:44


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.629241,1.314877,0.781671,03:32
1,0.660716,1.256529,0.781671,03:17
2,0.576824,1.193663,0.804582,03:35
3,0.497138,1.160173,0.803235,03:39


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.491431,1.192627,0.803235,03:26
1,0.502830,1.151319,0.812668,03:35
2,0.415567,1.141369,0.818059,03:29
3,0.350178,1.103785,0.819407,03:24


In [39]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.354612,1.166176,0.809973,03:16
1,0.389529,1.128959,0.820755,03:18
2,0.358228,1.079651,0.828841,03:26
3,0.293039,1.078379,0.830189,03:35


In [40]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.283412,1.147953,0.820755,03:31
1,0.339905,1.157306,0.830189,03:34
2,0.304252,1.108460,0.836927,03:26
3,0.246475,1.113789,0.836927,03:15


In [41]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.252536,1.116741,0.832884,03:38
1,0.312478,1.106091,0.835580,03:36
2,0.254409,1.146005,0.836927,03:27
3,0.222430,1.086820,0.839623,03:21


In [42]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.227749,1.162006,0.845013,03:34
1,0.278727,1.191556,0.838275,03:13
2,0.243214,1.210213,0.838275,03:29
3,0.202392,1.162865,0.840970,03:38


In [43]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.207375,1.172120,0.835580,03:23
1,0.247716,1.216843,0.839623,03:14
2,0.221243,1.178124,0.843666,03:22
3,0.178859,1.198529,0.840970,03:17


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.196051,1.113240,0.840970,03:36
1,0.194956,1.154222,0.840970,03:28


In [45]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.202127,1.173942,0.846361,03:41
1,0.187528,1.151056,0.843666,03:23


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.193455,1.194947,0.842318,03:56


In [47]:
# save the best model

learn.save_encoder('pubmed_cadec_askpatient_mixed_fold9')

# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[861, 4, 444, 983, 8, 8, 8, 8, 8, 240, 359, 359, 20, 20, 574, 439, 27, 27, 27, 750, 31, 31, 584, 129, 37, 37, 37, 37, 702, 491, 37, 37, 37, 37, 1025, 41, 49, 49, 49, 49, 49, 368, 853, 853, 517, 115, 646, 63, 426, 509, 71, 822, 76, 76, 512, 79, 265, 265, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 265, 79, 79, 79, 79, 79, 46, 228, 289, 80, 107, 491, 128, 85, 88, 352, 268, 268, 964, 91, 91, 92, 94, 94, 94, 94, 94, 94, 94, 94, 77, 598, 96, 594, 408, 98, 100, 100, 100, 100, 100, 877, 108, 108, 108, 108, 111, 111, 111, 111, 117, 117, 117, 117, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 773, 129, 343, 1026, 225, 268, 268, 137, 646, 499, 142, 148, 151, 151, 805, 980, 980, 321, 166, 166, 166, 166, 166, 166, 774, 172, 172, 172, 700, 79, 172, 172, 172, 214, 877, 624, 165, 904, 623, 182, 182, 182, 182, 183, 183, 183, 183, 183, 183, 184, 187, 187, 187, 188, 190, 190, 190, 289, 190, 190, 260, 197, 197, 202, 2

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

659
0.7609699769053118
